In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import pandas as pd


In [2]:
import sys
sys.path.append(f'../../src/python')
from grid_search import create_label, balance_unique_id, \
create_balanced_excluded, create_train_test_excluded, group_by_case, create_feature_label

In [3]:
df = pd.read_json(r"../../data/processed/pre-processed_2023_03_20.json")

In [4]:
label_name="phase2"
df1 = create_label(df, label_name)
df_unique = balance_unique_id(df1)
df_balanced_unique, df_balanced, df_excluded = create_balanced_excluded(df_unique, df1)
df_train, df_test, df_test1 = create_train_test_excluded(df_balanced, df_balanced_unique, df_excluded)
df_train_grouped, df_test_grouped, df_test1_grouped, df_excluded_grouped = group_by_case(df_train, df_test, df_test1, df_excluded)

Total decisions: 1583
0    1485
1      98
Name: label, dtype: int64
Balancing...
Total decisions: 196
Labels distribution: 
 0    98
1    98
Name: 0, dtype: int64
Training set shape: (157,) (157,)
Test set shape: (40,) (40,)
Creating df_train 1: 78 0: 80
Creating df_test1 1: 21 0: 20
Creating df_test concatenated with df_excluded with len: 1386
Creating df_test 1: 21 0: 1406
Grouping df_train by case_num 1: 78 0: 80
Grouping df_test by case_num 1: 21 0: 1406
Grouping df_test1 by case_num 1: 21 0: 20
Grouping df_excluded by case_num with len: 1386


In [5]:
X_train, y_train, X_test, y_test, X_test1, y_test1, X_test2, y_test2 = create_feature_label(df_train_grouped, df_test_grouped, df_test1_grouped, df_excluded_grouped)

In [43]:
stop_words = set(stopwords.words('english'))
tfidf = TfidfVectorizer(ngram_range=(2, 2), stop_words=stop_words)

X = tfidf.fit_transform(X_train)
feature_names = tfidf.get_feature_names_out()
scores = X.toarray()
print("\n\nScores : \n", scores)

# Getting top ranking features
sums = X.sum(axis = 0)
data1 = []
for col, term in enumerate(feature_names):
    data1.append((term, sums[0, col]))
ranking = pd.DataFrame(data1, columns=['term', 'rank'])
words = ranking.sort_values('rank', ascending=False)
print("\n\nTop 7 words by rank: \n", words.head(7))



Scores : 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


Top 7 words by rank: 
                           term      rank
430611         notifying party  6.282956
392278            market share  5.123169
391069    market investigation  3.918494
504549          product market  2.766597
522710  question questionnaire  2.758741
390109       market definition  2.494963
110759     commission decision  2.196066


In [49]:
# words.to_excel('../../../../output/tables/most_common_bigrams.xlsx')

In [47]:
words.head(20)

,term,rank
430611,notifying party,6.282956
392278,market share,5.123169
391069,market investigation,3.918494
504549,product market,2.766597
522710,question questionnaire,2.758741
390109,market definition,2.494963
110759,commission decision,2.196066
548537,reply question,2.181320
86081,case comp,2.115289
400473,merged entity,2.094990


In [48]:
words[words.term.str.contains('tfeu')]

,term,rank
659992,terminology tfeu,0.192810
661041,tfeu introduced,0.192810
661037,tfeu decision,0.192810
685149,union tfeu,0.189911
44092,article tfeu,0.106579
661058,tfeu unlikely,0.031603
661044,tfeu merger,0.026921
661050,tfeu relation,0.011572
169582,december tfeu,0.010902
455186,page tfeu,0.010902


In [7]:
df1[df1['text_clean'].str.contains('efficiency')]

,year,article_new,case_num,file,section_fin,len_pdf,sec_text,count,text_clean,count_text,phase2,wc,competition,label
5,2004,article6(1)(b),M.3439,\m3439_20040809_20310_en,Market Definition,27,A. General overview\ni) Pre-press printing tec...,11277,general overview pre press printing technology...,6274,0,0,0,0
49,2004,article6(1)(b),M.3516,\m3516_en,Competitive Assessment,9,15. Whilst the Portuguese energy sector is und...,2026,whilst portuguese energy sector undergoing res...,1060,0,0,0,0
94,2004,article6(1)(b),M.3605,\m3605_20041221_20310_en,Competitive Assessment,27,A. Relevant markets\n9. The parties claim th...,11262,relevant market party claim stage distinguishe...,5789,0,0,0,0
155,2005,article6(1)(b),M.3688,\m3688_20050318_20310_en,Competitive Assessment,11,A. Relevant product markets\n9. The parties...,3709,relevant product market party cid activity ove...,1930,0,0,0,0
273,2005,article6(1)(b),M.3856,\m3856_20050809_20310_en,Competitive Assessment,10,11. The creation of United Launch Alliance is ...,3046,creation united launch alliance likely direct ...,1508,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5285,2022,article6(1)(b),M.10619,\M_10619_8706541_447_3,Competitive Assessment,42,(2) SNAM is the holding company of the SNAM g...,11782,snam holding company snam group control snam r...,6192,0,0,0,0
5299,2022,article6(1)(b),M.10651,\M_10651_8486100_969_3,Market Definition,22,(7) Faerch and Paccor are both active i...,4856,faerch paccor active manufacture supply rigid ...,2668,0,0,0,0
5323,2022,article6(1)(b),M.10719,\M_10719_8502050_180_3,Competitive Assessment,22,5.1. Analytical framework\n(44) Article 2 ...,3239,analytical framework article merger regulation...,1788,0,0,0,0
5346,2022,article6(2),M.10506,\M_10506_8413340_1616_3,Competitive Assessment,95,4.1. Legal framework\n(101) When the parties...,21050,legal framework party concentration actual pot...,11799,0,1,1,0


In [9]:
print(df1['sec_text'][5346])

4.1.  Legal framework
(101)  When the parties to a concentration are actual or potential competitors in one or
more of the relevant product markets, the Transaction gives rise to horizontal effects.
The Commission Guidelines on the assessment of horizontal mergers under the
Council  Regulation  on  the  control  of  concentrations  between  undertakings  (the
‘Horizontal Merger Guidelines’)133 distinguish between two main ways in which
mergers may significantly impede effective competition, namely non-coordinated
and coordinated effects. In the present case, non-coordinated effects arise.
(102)  Non-coordinated  effects  may  significantly  impede  effective  competition  by
eliminating  important  competitive  constraints  on  one  or  more  firms,  which
consequently  would  have  increased  market  power.  The  Horizontal  Merger
Guidelines consider not only the direct loss of competition between the merging
firms, but also the reduction in competitive pressure on non-merging firms 